In [19]:
import boto3

s3 = boto3.Session(profile_name="data-prod").client("s3")

In [20]:
def s3_items_from_table(bucket, prefix):
    paginator = s3.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=bucket, Prefix=prefix)
    objects = []

    for page in pages:
        for item in page["Contents"]:
            objects.append(item["Key"])

    return objects

In [21]:
bucket = "heroes-dl-raw-prod"
s3_keys = s3_items_from_table(bucket=bucket, prefix="airbyte/sp_api/")

In [ ]:
print(len(s3_keys))
print(s3_keys[:5])

In [14]:
dt_from = "1970-01-01"
dt_to = "2023-07-25"

selected_entity = "ListFinancialEvents"

In [22]:
uris_filtered = []
for uri in s3_keys:
    if selected_entity in uri:
        aux = uri.split("/")[6]
        aux = aux.replace("p_ingestion_dt=", "")

        if dt_from <= aux <= dt_to:
            uris_filtered.append(uri)

print(f"Elements to delete for entity {selected_entity}: {len(uris_filtered)}")
print()

Elements to delete for entity ListFinancialEvents: 333



In [ ]:
uris_filtered[:10]

In [24]:
def s3_one_by_one(bucket, key):
    try:
        s3.delete_object(Bucket=bucket, Key=key)
        return f"key: >>{key}<< successfully deleted"
    except ValueError as e:
        return f"there was a problem with key: >>{key}<<, {e}"

In [ ]:
# deleting objects one by one
for i in uris_filtered:
    print(s3_one_by_one(bucket, i))